## 实验介绍

### 1.实验内容

本实验介绍SVD算法。并使用python实现该算法。

### 2.实验目标

通过本实验掌握SVD算法。

### 3.实验知识点

* SVD算法

### 4.实验环境

* python 3.6.5

### 5.预备知识

* Python编程基础

## 【原理】SVD是什么？ 
### 1.SVD是什么？

奇异值分解(singular value decomposition,SVD),是一种提取信息的方法。比如我们可以对记录用户关于餐馆观点的数据进行处理，并且从中提取出其背后的因素。这些因素可能会与餐馆的类别、烹饪时所用的某个特定配料，或其他任意对象一致。然后，我们就可以利用这些因素来估计人们对没有去过的餐馆的看法。

提取这些信息的方法就是奇异值分解法。


## 【原理】SVD能干什么？ 
### 2.SVD能干什么？

奇异值分解能够简约数据，去除噪声和冗余数据。其实它说白了也是一种降维方法，将数据映射到低维空间。我们之前学到的PCA算法也是数据降维的一种方法，那么二者有什么区别和联系呢？二者都是挖掘数据中的重要特征。PCA是基于SVD实现的，SVD比PCA更为稳定，SVD还可以应用到推荐系统上。

SVD的用处有很多，比如：LSA（隐性语义分析）、推荐系统、特征压缩（或称数据降维）。SVD可以理解为：将一个比较复杂的矩阵用更小更简单的3个子矩阵的相乘来表示，这3个小矩阵描述了大矩阵重要的特性。

## 【原理】奇异值分解 
### 3.奇异值分解

下面我们来回顾一下我们学过的矩阵知识，奇异值分解。对于一个m*n的矩阵，我们怎样才能描述这样普通的矩阵的重要特征呢？奇异值分解可以解决这个难题，它是一个能适用于任意的矩阵的一种分解的方法。奇异值分解将矩阵分解为3个子矩阵相乘的形式：

    A=U* Σ* VT

如果原始矩阵A是m行n列，那么U、Σ和VT就分别是m行m列、m行n列和n行n列。上述分解中会构建出一个矩阵Σ，该矩阵只有对角元素，其他元素均为0。另一个惯例就是，Σ的对角元素是从大到小排列的。这些对角元素称为奇异值（Singular Value），它们对应了原始数据集矩阵Data的奇异值。回想上一章的PCA，我们得到的是矩阵的特征值，它们告诉我们数据集中的重要特征。Σ中的奇异值也是如此。奇异值和特征值是有关系的。这里的奇异值就是矩阵A* A.T的特征值的平方根。

前面提到过，矩阵Σ只有从大到小排列的对角元素。在科学和工程中，一直存在这样一个普遍事实：在某个奇异值的数目（r个）之后，其他的奇异值都置为0。这就意味着数据集中仅有r个重要特征，而其余特征则都是噪声或冗余特征。

## 【原理】一个用例理解SVD 
### 4.一个用例理解SVD

下图给出的矩阵是由餐馆的菜和品菜师对这些菜的意见构成的。品菜师可以采用1到5之间的任意一个整数来对菜评级。如果品菜师没有尝过某道菜，则评级为0。
![](1_svd_algo.png)
拿到这个问题，最直观的一个思路流程就是：计算菜系的相似度->结合评分->对没吃过的菜系计算预测评分->预测评分排序->推荐前x个菜。

SVD做的改进就是将矩阵分解，从数据中构建出一个主题空间，再在该主题空间下计算相似度，提高了推荐效果。在上例中，对数据矩阵进行SVD处理，会得到两个奇异值。因此，有两个概念或主题与此数据集相关联，比如我们基于每个组的共同特征来命名，可能是美式BBQ和日式食品这二维。

如何才能将原始数据变换到上述新空间中呢？根据SVD分解得到的VT矩阵会将用户映射到BBQ/日式食品空间去。类似地，U矩阵会将餐馆的菜映射到BBQ/日式食品空间去。具体的推荐系统我们会在后面讲到。

目前我们只需要了解SVD在推荐系统中的作用就是对矩阵进行分解，将原始数据重构为只包含重要特征的数据矩阵

## 【实验】利用python实现SVD 
上节我们介绍了SVD是多么的厉害，那么该如何实现它呢？SVD的实现设计很多线性代数知识，我们不需要关注具体的奇异值分解细节，python的Numpy库为我们提供了linalg的线性代数工具。可以直接对矩阵进行奇异值分解。

我们先来了解一下如何利用该工具实现矩阵的SVD处理：

In [1]:
from numpy import *
u,sigma,vt = linalg.svd([[1,1],[7,7]])

接下来，我们可以逐个查看分解的矩阵

In [7]:
u

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [8]:
sigma

array([ 10.,   0.])

In [9]:
vt

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

矩阵sigma是以行向量array([10.,0.])返回，这是因为矩阵sigma除对角元素其他均为0，numpy仅返回对角元素，但一旦看到sigma就要知道它是一个矩阵，而不是array。

## 【实验】在大数据集上使用SVD算法
接下来，我们在更大的数据集上进行分解：

In [10]:
from numpy import *
from numpy import linalg as la
def loadExData():
    return[[1,1,1,0,0],
           [2,2,2,0,0],
           [1,1,1,0,0],
           [5,5,5,0,0],
           [1,1,0,2,2],
           [0,0,0,3,3],
           [0,0,0,1,1]]

if __name__ == '__main__':
    #测试svd奇异值分解
    data=loadExData()
    u,sigma,vt=la.svd(data)
    print("奇异值矩阵：\n",sigma)
    sig3=mat([[sigma[0],0,0],[0,sigma[1],0],[0,0,sigma[2]]])
    print("重构矩阵:\n",u[:,:3]*sig3*vt[:3,:])

奇异值矩阵：
 [  9.72140007e+00   5.29397912e+00   6.84226362e-01   1.76485425e-15
   1.07337736e-31]
重构矩阵:
 [[  1.00000000e+00   1.00000000e+00   1.00000000e+00  -3.06102944e-16
   -3.24716158e-16]
 [  2.00000000e+00   2.00000000e+00   2.00000000e+00   1.56507791e-16
    1.19281363e-16]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00   1.59285357e-15
    1.57424036e-15]
 [  5.00000000e+00   5.00000000e+00   5.00000000e+00  -1.16268486e-17
   -1.04909757e-16]
 [  1.00000000e+00   1.00000000e+00  -1.08845596e-15   2.00000000e+00
    2.00000000e+00]
 [  2.54588945e-16   1.60102837e-15  -1.68123482e-15   3.00000000e+00
    3.00000000e+00]
 [  1.30086946e-16   5.78900087e-16  -6.70962949e-16   1.00000000e+00
    1.00000000e+00]]


我们来逐个分析，sigma矩阵中，可以明显看到，前三个奇异值比其他的值大了很多，我们只保留了前三个值。那为什么对于奇异值的数目只保留三个呢？其中一个典型的做法是保留矩阵中90%的能量信息。另一种方法是，当矩阵上有上万的奇异值时，那么就保留前面的2000或3000个。

在这里，我们保留了前三个奇异值，那么我们的原始数据集就可以用如下结果来近似：
![](2_svd_algo.png)
近似过程如下图：
![](3_svd_algo.png)
根据这个方案来重构原始数据。也就是代码中的：
        
        u[:,:3]*sig3*vt[:3,:]
由于Sig3仅为3×3的矩阵，因而我们只需使用矩阵U的前三列和VT的前三行。

现在我们已经可以使用SVD对一个大矩阵进行处理。

## 实验总结

本实验中我们介绍了SVD是一种强大的降维工具，我们可以利用SVD来逼近矩阵并从中提取重要特征。通过保留矩阵80-90%的能量，就可以得到重要的特征并去掉噪声。介绍了如何使用python实现该算法，您应该能达到以下两个目标：

    1. 掌握SVD算法。
    2. 学会实现相应算法。

##  参考文献及延伸阅读 

### 参考资料：

1.哈林顿，李锐. 机器学习实战 : Machine learning in action[M]. 人民邮电出版社, 2013.  
2.周志华. 机器学习:Machine learning[M]. 清华大学出版社, 2016.

### 延伸阅读：

1.李航. 统计学习方法[M]. 清华大学出版社, 2012.